In [3]:
import os
import sys
import random
import time
from random import seed, randint
import argparse
import platform
from datetime import datetime
import imp
import numpy as np
import fileinput
from itertools import product
import pandas as pd
from scipy.interpolate import griddata
from scipy.interpolate import interp2d
import seaborn as sns
from os import listdir

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import griddata
import matplotlib as mpl
sys.path.insert(0,'..')
# from notebookFunctions import *
# from .. import notebookFunctions

%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6.180)    #golden ratio
# %matplotlib notebook
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '/Users/weilu/openmmawsem/')

from openmmawsem import *

In [2]:
os.chdir("/Users/weilu/openmmawsem/_local/multi_chain_1brr/openmm/")

In [4]:
input_pdb_filename, cleaned_pdb_filename =prepare_pdb("1brr.pdb", "ABC")
ensure_atom_order(input_pdb_filename, quiet=0)

Reorder atom position
Original, Changed to
2697 CA ,  2697 CA
2698 CB ,  2699 C
2699 C ,  2700 O
2700 O ,  2698 CB


In [7]:
ensure_atom_order(input_pdb_filename, quiet=0)

In [8]:
pdb_filename = input_pdb_filename
pdb = PDBFile(pdb_filename)
forcefield = ForceField("../../../awsem.xml")
forcefield.createSystem(pdb.topology)

resi = [x.residue.index for x in list(pdb.topology.atoms())]
nres = pdb.topology.getNumResidues()
natoms = pdb.topology.getNumAtoms()
residues = list(pdb.topology.residues())
atom_lists,res_type=build_lists_of_atoms_2(nres, residues, pdb.topology.atoms())

In [9]:
n =atom_lists['n']
h =atom_lists['h']
ca=atom_lists['ca']
c =atom_lists['c']
o =atom_lists['o']
cb=atom_lists['cb']

In [10]:
nres

693

In [11]:
len(n)

693

In [48]:
pdb_filename = "crystal_structure-cleaned.pdb"
pdb = PDBFile(pdb_filename)
residues = list(pdb.topology.residues())
chain_name = 'AB'

In [49]:
a = pdb.topology.chains

In [54]:
b = list(a())[0]

In [ ]:
pdb.topology.

In [63]:
c = list(b.residues())[0]

In [75]:
d = list(c.atoms())[4]

In [74]:
c

<Residue 0 (GLN) of chain 0>

In [81]:
pdb_file = "crystal_structure-cleaned.pdb"
parser = PDBParser()
structure = parser.get_structure('X', pdb_file)

In [83]:
a = structure[0]

In [87]:
b = list(a.get_chains())[0]

In [90]:
c = list(b.get_residues())[0]

In [92]:
c.

(' ', 1, ' ')

In [ ]:
    def read_reference_structure_for_q_calculation_2(self, pdb_file, min_seq_sep=3, max_seq_sep=np.inf, contact_threshold=0.8*nanometers):
        # default use all chains in pdb file.
        parser = PDBParser()
        structure = parser.get_structure('X', pdb_file)
        model = structure[0]
        chain_start = 0
        count = 0
        for chain in model.get_chains():
            chain_start += count
            count = 0
            for i, residue_i in enumerate(chain.get_residues()):
                count += 1
                #  print(i, residue_i)
                 for j, residue_j in enumerate(chain.get_residues()):
                    ca_list = []
                    cb_list = []
                    atom_list_i = []
                    atom_list_j = []
                    if i-j >= min_seq_sep and i-j <= max_seq_sep:  # taking the signed value to avoid double counting
                        ca_i = residue_i['CA']
                        ca_list.append(ca_i)
                        atom_list_i.append(ca_i)
                        ca_j = residue_j['CA']
                        ca_list.append(ca_j)
                        atom_list_j.append(ca_j)
                        if not residue_i.get_resname() == "GLY":
                            cb_i = residue_i['CB']
                            cb_list.append(cb_i)
                            atom_list_i.append(cb_i)
                        if not residue_j.get_resname() == "GLY":
                            cb_j = residue_j['CB']
                            cb_list.append(cb_j)
                            atom_list_j.append(cb_j)
                        for atom_i, atom_j in product(atom_list_i, atom_list_j):
                            r_ijN = abs(atom_i - atom_j)/10.0*nanometers # convert to nm
                            if r_ijN <= contact_threshold:
                                sigma_ij = 0.1*abs(i-j)**0.15 # 0.1 nm = 1 A
                                gamma_ij = 1.0
                                if atom_i in ca_list:
                                    i_index = self.ca[i+chain_start]
                                if atom_i in cb_list:
                                    i_index = self.cb[i+chain_start]
                                if atom_j in ca_list:
                                    j_index = self.ca[j+chain_start]
                                if atom_j in cb_list:
                                    j_index = self.cb[j+chain_start]
                                structure_interaction = [i_index, j_index, [gamma_ij, r_ijN, sigma_ij]]
                                structure_interactions.append(structure_interaction)

        return structure_interactions



In [ ]:
    def read_reference_structure_for_q_calculation_2(self, pdb_file, min_seq_sep=3, max_seq_sep=np.inf, contact_threshold=0.8*nanometers):
        # default use all chains in pdb file.
        pdb = PDBFile(pdb_file)
        for chain in pdb.topology.chains():
            for residue_i in chain.residues():
                i = residue_i.index
                #  print(i, residue_i)
                for residue_j in chain.residues():
                    j = residue_j.index
                    ca_list = []
                    cb_list = []
                    atom_list_i = []
                    atom_list_j = []
                    if i-j >= min_seq_sep and i-j <= max_seq_sep:  # taking the signed value to avoid double counting
                        ca_i = residue_i['CA']
                        ca_list.append(ca_i)
                        atom_list_i.append(ca_i)
                        ca_j = residue_j['CA']
                        ca_list.append(ca_j)
                        atom_list_j.append(ca_j)
                        if not residue_i.get_resname() == "GLY":
                            cb_i = residue_i['CB']
                            cb_list.append(cb_i)
                            atom_list_i.append(cb_i)
                        if not residue_j.get_resname() == "GLY":
                            cb_j = residue_j['CB']
                            cb_list.append(cb_j)
                            atom_list_j.append(cb_j)
                        for atom_i, atom_j in product(atom_list_i, atom_list_j):
                            r_ijN = abs(atom_i - atom_j)/10.0*nanometers # convert to nm
                            if r_ijN <= contact_threshold:
                                sigma_ij = 0.1*abs(i-j)**0.15 # 0.1 nm = 1 A
                                gamma_ij = 1.0
                                if atom_i in ca_list:
                                    i_index = self.ca[i]
                                if atom_i in cb_list:
                                    i_index = self.cb[i]
                                if atom_j in ca_list:
                                    j_index = self.ca[j]
                                if atom_j in cb_list:
                                    j_index = self.cb[j]
                                structure_interaction = [i_index, j_index, [gamma_ij, r_ijN, sigma_ij]]
                                structure_interactions.append(structure_interaction)

        return structure_interactions


In [ ]:
pdb.topology.

In [61]:
for chain in pdb.topology.chains():
    for i, residue_i in enumerate(chain.residues()):
        print(i, residue_i)
        for j, residue_j in enumerate(chain.residues()):
    #         print(i,j)
            pass

0 <Residue 0 (GLN) of chain 0>
1 <Residue 1 (ILE) of chain 0>
2 <Residue 2 (THR) of chain 0>
3 <Residue 3 (GLY) of chain 0>
4 <Residue 4 (ARG) of chain 0>
5 <Residue 5 (PRO) of chain 0>
6 <Residue 6 (GLU) of chain 0>
7 <Residue 7 (TRP) of chain 0>
8 <Residue 8 (ILE) of chain 0>
9 <Residue 9 (TRP) of chain 0>
10 <Residue 10 (LEU) of chain 0>
11 <Residue 11 (ALA) of chain 0>
12 <Residue 12 (LEU) of chain 0>
13 <Residue 13 (GLY) of chain 0>
14 <Residue 14 (THR) of chain 0>
15 <Residue 15 (ALA) of chain 0>
16 <Residue 16 (LEU) of chain 0>
17 <Residue 17 (MET) of chain 0>
18 <Residue 18 (GLY) of chain 0>
19 <Residue 19 (LEU) of chain 0>
20 <Residue 20 (GLY) of chain 0>
21 <Residue 21 (THR) of chain 0>
22 <Residue 22 (LEU) of chain 0>
23 <Residue 23 (TYR) of chain 0>
24 <Residue 24 (PHE) of chain 0>
25 <Residue 25 (LEU) of chain 0>
26 <Residue 26 (VAL) of chain 0>
27 <Residue 27 (LYS) of chain 0>
28 <Residue 28 (GLY) of chain 0>
29 <Residue 29 (MET) of chain 0>
30 <Residue 30 (GLY) of chain 

67 <Residue 297 (VAL) of chain 1>
68 <Residue 298 (PRO) of chain 1>
69 <Residue 299 (PHE) of chain 1>
70 <Residue 300 (GLY) of chain 1>
71 <Residue 301 (GLY) of chain 1>
72 <Residue 302 (GLU) of chain 1>
73 <Residue 303 (GLN) of chain 1>
74 <Residue 304 (ASN) of chain 1>
75 <Residue 305 (PRO) of chain 1>
76 <Residue 306 (ILE) of chain 1>
77 <Residue 307 (TYR) of chain 1>
78 <Residue 308 (TRP) of chain 1>
79 <Residue 309 (ALA) of chain 1>
80 <Residue 310 (ARG) of chain 1>
81 <Residue 311 (TYR) of chain 1>
82 <Residue 312 (ALA) of chain 1>
83 <Residue 313 (ASP) of chain 1>
84 <Residue 314 (TRP) of chain 1>
85 <Residue 315 (LEU) of chain 1>
86 <Residue 316 (PHE) of chain 1>
87 <Residue 317 (THR) of chain 1>
88 <Residue 318 (THR) of chain 1>
89 <Residue 319 (PRO) of chain 1>
90 <Residue 320 (LEU) of chain 1>
91 <Residue 321 (LEU) of chain 1>
92 <Residue 322 (LEU) of chain 1>
93 <Residue 323 (LEU) of chain 1>
94 <Residue 324 (ASP) of chain 1>
95 <Residue 325 (LEU) of chain 1>
96 <Residue 32

In [13]:
a = list(pdb.topology.residues())

In [18]:
a1 = a[0]

In [21]:
a12 = a1.chain

In [39]:
a = getSeqFromCleanPdb(pdb_filename, chains="ABC")

In [40]:
len(a)

693

In [35]:
identify_terminal_residues(input_pdb_filename)

{'A': (1, 230), 'B': (1, 231), 'C': (1, 232)}

In [32]:
def get_chain_starts_and_ends(all_res):
    chain_starts = []
    chain_ends = []
    pos = -1
    for i, res in enumerate(all_res):
        if res.chain.index != pos:
            chain_starts.append(i)
            pos = res.chain.index
            if i > 0:
                chain_ends.append(i-1)
    chain_ends.append(len(a)-1)
    return chain_starts, chain_ends

In [33]:
get_chain_starts_and_ends(residues)

([0, 230, 461], [229, 460, 692])

In [59]:
pdb_file = "crystal_structure-cleaned.pdb"
chain_name = 'AB'
structure_interactions = []
parser = PDBParser()
structure = parser.get_structure('X', pdb_file)
# chain = structure[0][chain_name]
# residues = [x for x in chain]
for chain in chain_name:
    residues = [x for x in structure[0][chain]]
    for i, residue_i in enumerate(residues):
        print(i, residue_i)
        for j, residue_j in enumerate(residues):
    #         print(i,j)
            pass

0 <Residue GLN het=  resseq=1 icode= >
1 <Residue ILE het=  resseq=2 icode= >
2 <Residue THR het=  resseq=3 icode= >
3 <Residue GLY het=  resseq=4 icode= >
4 <Residue ARG het=  resseq=5 icode= >
5 <Residue PRO het=  resseq=6 icode= >
6 <Residue GLU het=  resseq=7 icode= >
7 <Residue TRP het=  resseq=8 icode= >
8 <Residue ILE het=  resseq=9 icode= >
9 <Residue TRP het=  resseq=10 icode= >
10 <Residue LEU het=  resseq=11 icode= >
11 <Residue ALA het=  resseq=12 icode= >
12 <Residue LEU het=  resseq=13 icode= >
13 <Residue GLY het=  resseq=14 icode= >
14 <Residue THR het=  resseq=15 icode= >
15 <Residue ALA het=  resseq=16 icode= >
16 <Residue LEU het=  resseq=17 icode= >
17 <Residue MET het=  resseq=18 icode= >
18 <Residue GLY het=  resseq=19 icode= >
19 <Residue LEU het=  resseq=20 icode= >
20 <Residue GLY het=  resseq=21 icode= >
21 <Residue THR het=  resseq=22 icode= >
22 <Residue LEU het=  resseq=23 icode= >
23 <Residue TYR het=  resseq=24 icode= >
24 <Residue PHE het=  resseq=25 ico

80 <Residue ARG het=  resseq=81 icode= >
81 <Residue TYR het=  resseq=82 icode= >
82 <Residue ALA het=  resseq=83 icode= >
83 <Residue ASP het=  resseq=84 icode= >
84 <Residue TRP het=  resseq=85 icode= >
85 <Residue LEU het=  resseq=86 icode= >
86 <Residue PHE het=  resseq=87 icode= >
87 <Residue THR het=  resseq=88 icode= >
88 <Residue THR het=  resseq=89 icode= >
89 <Residue PRO het=  resseq=90 icode= >
90 <Residue LEU het=  resseq=91 icode= >
91 <Residue LEU het=  resseq=92 icode= >
92 <Residue LEU het=  resseq=93 icode= >
93 <Residue LEU het=  resseq=94 icode= >
94 <Residue ASP het=  resseq=95 icode= >
95 <Residue LEU het=  resseq=96 icode= >
96 <Residue ALA het=  resseq=97 icode= >
97 <Residue LEU het=  resseq=98 icode= >
98 <Residue LEU het=  resseq=99 icode= >
99 <Residue VAL het=  resseq=100 icode= >
100 <Residue ASP het=  resseq=101 icode= >
101 <Residue ALA het=  resseq=102 icode= >
102 <Residue ASP het=  resseq=103 icode= >
103 <Residue GLN het=  resseq=104 icode= >
104 <Re

In [5]:
pdb_trajectory = read_trajectory_pdb_positions("openmmMovie.pdb")
oa = OpenMMAWSEMSystem(input_pdb_filename, chains='ABC', k_awsem=1.0, xml_filename="../../../awsem.xml") # k_awsem is an overall scaling factor that will affect the relevant temperature scales

# apply forces
# forceGroupTable_Rev = {11:"Con", 12:"Chain", 13:"Chi", 14:"Excluded", 15:"Rama", 16:"Direct", 
#                   17:"Burial", 18:"Mediated", 19:"Fragment"}
forceGroupTable = {"Con":11, "Chain":12, "Chi":13, "Excluded":14, "Rama":15, "Direct":16,
                    "Burial":17, "Mediated":18, "Contact":18, "Fragment":19, "Total":list(range(11, 20)),
                    "Water":[16, 18], "Q":1}
forces = [
#     oa.q_value("crystal_structure-cleaned.pdb", "ABC"),
#     oa.con_term(),
    oa.chain_term(),
#     oa.chi_term(),
#     oa.excl_term(),
#     oa.rama_term(),
#     oa.rama_proline_term(),
#     oa.direct_term(),
#     oa.burial_term(),
#     oa.mediated_term(),
#     oa.fragment_memory_term(frag_location_pre="./")
]
oa.addForcesWithDefaultForceGroup(forces)

# start simulation
collision_rate = 5.0 / picoseconds

integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
simulation = Simulation(oa.pdb.topology, oa.system, integrator, Platform.getPlatformByName("CPU"))

hi
hi
hi2
hi
hi


In [6]:
showEnergy = ["Q", "Con", "Chain", "Chi", "Excluded", "Rama", "Water", "Burial", "Fragment", "Total"]
# print("Steps", *showEnergy)
print(" ".join(["{0:<8s}".format(i) for i in ["Steps"] + showEnergy]))
for step, pdb in enumerate(pdb_trajectory):
    simulation.context.setPositions(pdb.positions)
    e = []
    for term in showEnergy:
        if type(forceGroupTable[term]) == list:
            g = set(forceGroupTable[term])
        elif forceGroupTable[term] == -1:
            g = -1
        else:
            g = {forceGroupTable[term]}
        state = simulation.context.getState(getEnergy=True, groups=g)
        termEnergy = state.getPotentialEnergy().value_in_unit(kilocalories_per_mole)
        e.append(termEnergy)
#     print(*e)
    print(" ".join([f"{step:<8}"] +  ["{0:<8.2f}".format(i) for i in e]))
#         print(forceGroupTable[term], state.getPotentialEnergy().value_in_unit(kilocalories_per_mole))

Steps    Q        Con      Chain    Chi      Excluded Rama     Water    Burial   Fragment Total   
0        0.00     0.00     454725.48 0.00     0.00     0.00     0.00     0.00     0.00     454725.48
1        0.00     0.00     714121.74 0.00     0.00     0.00     0.00     0.00     0.00     714121.74
2        0.00     0.00     618193.42 0.00     0.00     0.00     0.00     0.00     0.00     618193.42
3        0.00     0.00     788246.49 0.00     0.00     0.00     0.00     0.00     0.00     788246.49
4        0.00     0.00     709550.92 0.00     0.00     0.00     0.00     0.00     0.00     709550.92
5        0.00     0.00     648436.60 0.00     0.00     0.00     0.00     0.00     0.00     648436.60
6        0.00     0.00     650457.36 0.00     0.00     0.00     0.00     0.00     0.00     650457.36
7        0.00     0.00     601024.96 0.00     0.00     0.00     0.00     0.00     0.00     601024.96
8        0.00     0.00     503666.88 0.00     0.00     0.00     0.00     0.00     0.00     50

In [1]:
chain

NameError: name 'chain' is not defined

In [88]:
x = 0.0
print(f"---{x:8.3f}---")
print(len(f"{x:8.3f}"))
print(f"---{1:6.2f}---")

---   0.000---
8
---  1.00---


In [ ]:

line = "{type:4}  {serialNumber:5} {AtomName:4} {ReidueName:3} {Chain:1}{ResId:4} {x:8.3f}{y:8.3f}{z:8.3f}{1:6.2f}{0:6.2f}    {element} \n"






In [4]:
file = "tmp.pdb"
fileTo = "test_tmp.pdb"
location = "/Users/weilu/Research/server/nov_2018/multi_chain/openmm/"
os.chdir(location)
i = 0
print(f"---{i:4}---")
def fixPDB(file="tmp.pdb", fileTo="test_tmp.pdb"):
#     chain_length = [230, 461, 693, 0]
#     chain = ["A","B","C"]
    chain_length = [42, 84, 0]
    chain = ["A","B"]
    pos = 0
    residue = 1
    with open(fileTo, "w") as out:
        with open(file, "r") as f:
            for line in f:
                tmp = list(line)
                try:
                    residue = int(line[22:26])
                except:
                    pass
                if residue > chain_length[pos]:
                    pos += 1
                    out.write("TER \n")
                i = residue-chain_length[pos-1]
                r = f"{i:4}"
                if len(tmp) > 21:
                    tmp[21] = chain[pos]
                    tmp[22:26] = r
                new_line = "".join(tmp)
                
                out.write(new_line)
a = open(location + "lammps_movie.pdb").read().split("END")
os.system("rm openmmMovie.pdb")
os.system(f"echo 'REMARK converted from awsem lammps output' >> openmmMovie.pdb")
for i in range(10):
    with open("tmp.pdb", "w") as out:
        out.write(a[i])
    fixPDB()
    input_pdb_filename, cleaned_pdb_filename =prepare_pdb("test_tmp.pdb", "ABC")
    os.system(f"echo 'MODEL  {i+1}' >> openmmMovie.pdb")
    os.system("cat test_tmp-openmmawsem.pdb >> openmmMovie.pdb")

fixPDB()

---   0---


In [5]:
parser = PDBParser()
structure = parser.get_structure('X', "/Users/weilu/openmmawsem/_local/gamma_secretase/crystal_structure.pdb")

/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10616.
  PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 10887.
  PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 10930.
  PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 10973.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 109

/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 11379.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 11379.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 11380.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or

/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 12479.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 12479.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 12480.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or

/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 13790.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing occupancy at line 13791.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or missing B factor at line 13791.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  % message, PDBConstructionWarning)
/Users/weilu/anaconda3/lib/python3.6/site-packages/Bio/PDB/PDBParser.py:289: PDBConstructionWarning: PDBConstructionException: Invalid or 

In [7]:
list(structure.get_chains())

[<Chain id=A>, <Chain id=B>, <Chain id=C>, <Chain id=D>, <Chain id= >]

In [19]:
a = "ABCD AaaBC "

In [6]:
a = pd.read_csv("/Users/weilu/Dropbox/Comp557/Pacwar/python/submission/2018_nov27_3.csv", index_col=0)

In [9]:
a.columns

Index(['gene1', 'score'], dtype='object')

In [11]:
for i, line in a.iterrows():
    print(line["gene1"])

03100000001000003333311121121112221122131131131131
03130000001000003333312121121112221122131131131131
03100000001000003333311121121112221122131130131131
00100000001000003333121111111111111121131133121131
03130000001000003333311121121112221122131131131131
03100000001000003333311121121111221122131130131131
00100000001000003333121111111111111111131133121331
03130000001000003333312121121112221122131133221131
03100000001000003333311121121111221122131131131131
03100000001000003333311121121112221122131133231131
03100000001000003333311121121111221122131133121131
00100000001000000333112111111111111121131131131131
03100000001000003333311121121111221122131133231131
00100000001000000333121111111111111121131131131131
03130000001000003333311121121112221122131130131131
03130000001000003333312121121112221122131130131131
03130000001000003333313121121111221122131130131131
03100000001000003333311121121112221122131133221131
03130000001000003333312121121112221122131133231131
0313000000100000333331112112111

In [22]:
''.join(sorted(set(a.upper().replace(" ", ""))))

'ABCD'